In [1]:
import { config } from "https://deno.land/x/dotenv/mod.ts";
import {z} from 'https://deno.land/x/zod@v3.23.8/mod.ts';
import { StructuredTool } from "https://esm.sh/@langchain/core/tools";
import { formatToOpenAITool } from "https://esm.sh/@langchain/openai";
import OpenAI from "https://deno.land/x/openai@v4.51.0/mod.ts";

const env = await config(); 

In [2]:
const OPENAI_API_KEY = env.OPENAI_API_KEY

In [3]:
import { ChatOpenAI } from "https://esm.sh/@langchain/openai";

In [4]:
const openai_client = new OpenAI({apiKey:OPENAI_API_KEY})

In [5]:
const client = new ChatOpenAI({apiKey:OPENAI_API_KEY})

In [6]:
await client.invoke("hi there")

g {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello! How can I assist you today?",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 9, promptTokens: 9, totalTokens: 18 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: [],
  usage_metadata: { input_tokens: 9, output_tokens: 9, total_tokens: 18 }
}

In [26]:
class AddNums extends StructuredTool {
    name = "add_numbers";
    description = "add two numbers together";

    schema =  z.object({
        num_1: z.number().describe("the first number to add"),
        num_2: z.number().describe("the second number to add"),
    })

    constructor() {
        super();
    }

    _call(input: z.infer<typeof this['schema']>): Promise<string> {
        return Promise.resolve(JSON.stringify(input, null, 2));
    } 
}

In [60]:
const addNums = new AddNums()

In [27]:
const tool = new AddNums()

In [28]:
tool._call({num_1: 3, num_2: 4}).then(console.log)

{
  "num_1": 3,
  "num_2": 4
}


Promise { undefined }

In [29]:
tool.schema.describe()

ZodObject {
  _type: undefined,
  _output: undefined,
  _input: undefined,
  _def: {
    shape: [Function: shape],
    unknownKeys: "strip",
    catchall: ZodNever {
      _type: undefined,
      _output: undefined,
      _input: undefined,
      _def: { typeName: "ZodNever" },
      spa: [Function: bound safeParseAsync] AsyncFunction,
      parse: [Function: bound parse],
      safeParse: [Function: bound safeParse],
      parseAsync: [Function: bound parseAsync] AsyncFunction,
      safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
      refine: [Function: bound refine],
      refinement: [Function: bound refinement],
      superRefine: [Function: bound superRefine],
      optional: [Function: bound optional],
      nullable: [Function: bound nullable],
      nullish: [Function: bound nullish],
      array: [Function: bound array],
      promise: [Function: bound promise],
      or: [Function: bound or],
      and: [Function: bound and],
      transform: [Function: boun

In [30]:
typeof(3)

"number"

In [59]:
addNums

ReferenceError: addNums is not defined

In [31]:
const addTool = formatToOpenAITool(new AddNums())

In [32]:
addTool

{
  type: "function",
  function: {
    name: "add_numbers",
    description: "add two numbers together",
    parameters: {
      type: "object",
      properties: {
        num_1: { type: "number", description: "the first number to add" },
        num_2: { type: "number", description: "the second number to add" }
      },
      required: [ "num_1", "num_2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

In [33]:
await openai_client.chat.completions.create({
      messages: [{ role: 'user', content: "hello tell me a joke" }],
      model: 'gpt-3.5-turbo-0125',
    });

{
  id: "chatcmpl-9ba3lMgido8bg7cUCH38nEner52KZ",
  object: "chat.completion",
  created: 1718743777,
  model: "gpt-3.5-turbo-0125",
  choices: [
    {
      index: 0,
      message: {
        role: "assistant",
        content: "Sure! Here's a joke for you:\n" +
          "\n" +
          "Why couldn't the bicycle find its way home?\n" +
          "\n" +
          "Because it lost its beari"... 4 more characters
      },
      logprobs: null,
      finish_reason: "stop"
    }
  ],
  usage: { prompt_tokens: 12, completion_tokens: 25, total_tokens: 37 },
  system_fingerprint: null
}

In [34]:
addTool

{
  type: "function",
  function: {
    name: "add_numbers",
    description: "add two numbers together",
    parameters: {
      type: "object",
      properties: {
        num_1: { type: "number", description: "the first number to add" },
        num_2: { type: "number", description: "the second number to add" }
      },
      required: [ "num_1", "num_2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

In [35]:
let tools = [addTool]

In [36]:
tools[0]

{
  type: "function",
  function: {
    name: "add_numbers",
    description: "add two numbers together",
    parameters: {
      type: "object",
      properties: {
        num_1: { type: "number", description: "the first number to add" },
        num_2: { type: "number", description: "the second number to add" }
      },
      required: [ "num_1", "num_2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

In [38]:
let res = await openai_client.chat.completions.create({
    model: 'gpt-3.5-turbo-0125',
    messages: [{ role: 'user', content: "Please add the numbers 3 and 50000" }],
    tools: tools
    });

In [54]:
let responseMessage = res.choices[0].message

In [57]:
responseMessage.tool_calls

[
  {
    id: "call_eErdX74hXIgvHu2XMBzzYMop",
    type: "function",
    function: { name: "add_numbers", arguments: '{"num_1":3,"num_2":50000}' }
  }
]

In [68]:
let params
if (responseMessage.tool_calls) {
    params = JSON.parse(responseMessage.tool_calls[0].function.arguments);
    console.log(params)
}

{ num_1: 3, num_2: 50000 }


In [69]:
params

{ num_1: 3, num_2: 50000 }

In [80]:
params

{ num_1: 3, num_2: 50000 }

In [88]:
function addingMachine ({num_1, num_2}) {
    return num_1 + num_2
}

In [89]:
console.log(addingMachine(params))

50003


In [53]:
JSON.parse(res.choices[0].message.tool_calls[0].function.arguments)

{ num_1: 3, num_2: 50000 }